In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50     # steepness factor
num_blocks = 10000 # number of expression blocks
num_data = 9100    # number of data
test_size = 100    # number of test size
val_size = 1000    # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [23]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


  1%|▊                                                                               | 1/100 [01:01<1:42:13, 61.96s/it]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


  2%|█▌                                                                              | 2/100 [02:03<1:40:23, 61.46s/it]

...

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [1:40:41<01:01, 61.01s/it]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:41:43<00:00, 61.03s/it]


ValueError: All arrays must be of the same length

## Heuristic - Round

In [25]:
from src.heuristic import naive_round

In [26]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


  0%|                                                                                          | 0/100 [01:04<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'items'

## Heuristic - N1

In [ ]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 2509939.02
Epoch 0, Iters 125, Validation Loss: 23547784.62
Epoch 1, Iters 250, Validation Loss: 1058362.64
Epoch 2, Iters 375, Validation Loss: 9029237344.00
Epoch 3, Iters 500, Validation Loss: 2525781.12
Epoch 4, Iters 625, Validation Loss: 763676.33
Epoch 5, Iters 750, Validation Loss: 2161524.15
Epoch 6, Iters 875, Validation Loss: 9943107776.00
Epoch 7, Iters 1000, Validation Loss: 395674504.00
Epoch 8, Iters 1125, Validation Loss: 1280656.49
Epoch 9, Iters 1250, Validation Loss: 816457.28
Epoch 10, Iters 1375, Validation Loss: 574433.58
Epoch 11, Iters 1500, Validation Loss: 642471.21
Epoch 12, Iters 1625, Validation Loss: 1408512982617.41
Epoch 13, Iters 1750, Validation Loss: 221375180800.00
Epoch 14, Iters 1875, Validation Loss: 645202.55
Epoch 15, Iters 2000, Validation Loss: 386957.65
Epoch 16, Iters 2125, Validation Loss: 307864.66
Epoch 17, Iters 2250, Validation Loss: 301432.22
Epoch 18, Iters 2375, Validation Loss: 790172.31
Epoch 19, 

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:21<00:00,  1.42s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    63394.521176       1886.499878      0.044669
std     22387.131319       2978.950012      0.005770
min     30213.719813          0.000000      0.010129
25%     41690.205890          0.000000      0.043941
50%     60144.230399          0.000000      0.044718
75%     84490.023807       3293.265869      0.046077
max    103941.439708      12096.487408      0.052395
Number of infeasible solution: 45


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 2261953.14
Epoch 0, Iters 125, Validation Loss: 5502742.38
Epoch 1, Iters 250, Validation Loss: 4919663.27
Epoch 2, Iters 375, Validation Loss: 8745273.75
Epoch 3, Iters 500, Validation Loss: 1767600.40
Epoch 4, Iters 625, Validation Loss: 978680.83
Epoch 5, Iters 750, Validation Loss: 1351430.10
Epoch 6, Iters 875, Validation Loss: 1245172.36
Epoch 7, Iters 1000, Validation Loss: 4438008.52
Epoch 8, Iters 1125, Validation Loss: 563766.44
Epoch 9, Iters 1250, Validation Loss: 591608.92
Epoch 10, Iters 1375, Validation Loss: 453391.23
Epoch 11, Iters 1500, Validation Loss: 807288.18
Epoch 12, Iters 1625, Validation Loss: 364644.39
Epoch 13, Iters 1750, Validation Loss: 441930.20
Epoch 14, Iters 1875, Validation Loss: 755670.53
Epoch 15, Iters 2000, Validation Loss: 415722.93
Epoch 16, Iters 2125, Validation Loss: 370731.40
Epoch 17, Iters 2250, Validation Loss: 956846.82
Epoch 18, Iters 2375, Validation Loss: 368800.33
Epoch 19, Iters 2500, Validation 

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lt_50-10000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.32s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    66890.890298       1035.811869      0.044335
std     44878.364357       2045.622608      0.004318
min     31365.948191          0.000000      0.016465
25%     39576.275118          0.000000      0.043772
50%     48003.272532          0.000000      0.044270
75%     74269.501782       1304.634400      0.044901
max    220276.080528       9271.844791      0.052884
Number of infeasible solution: 32


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 200  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 4465928.39
Epoch 1, Validation Loss: 806492.54
Epoch 2, Validation Loss: 1400804.34
Epoch 3, Validation Loss: 836137.80
Epoch 4, Validation Loss: 1132050.46
Epoch 5, Validation Loss: 932114.68
Epoch 6, Validation Loss: 1877306.24
Epoch 7, Validation Loss: 776943.61
Epoch 8, Validation Loss: 2467769.39
Epoch 9, Validation Loss: 747233.48
Epoch 10, Validation Loss: 771249.11
Epoch 11, Validation Loss: 712385.04
Epoch 12, Validation Loss: 753251.93
Epoch 13, Validation Loss: 576132.10
Epoch 14, Validation Loss: 560900.38
Epoch 15, Validation Loss: 539467.83
Epoch 16, Validation Loss: 1021170.49
Epoch 17, Validation Loss: 557530.21
Epoch 18, Validation Loss: 509458.69
Epoch 19, Validation Loss: 815782.66
Epoch 20, Validation Loss: 482423.45
Epoch 21, Validation Loss: 659804.84
Epoch 22, Validation Loss: 639800.97
Epoch 23, Validation Loss: 501593.99
Epoch 24, Validation Loss: 900555.49
Epoch 25, Validation Loss: 1437293.05
Epoch 26, Validation Loss: 495750.32
Epoc

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-1000.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:48<00:00,  1.69s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    58414.161239        783.294925      0.000745
std     33735.526360       1776.636885      0.000481
min     17820.777149          0.000000      0.000000
25%     29810.740259          0.000000      0.000000
50%     54241.666360          0.000000      0.000999
75%     76504.749923        182.742361      0.001004
max    142826.237652       8907.434940      0.002000
Number of infeasible solution: 26


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2270309.39
Epoch 1, Validation Loss: 607560.49
Epoch 2, Validation Loss: 562869.86
Epoch 3, Validation Loss: 545407.88
Epoch 4, Validation Loss: 529394.04
Epoch 5, Validation Loss: 548448.44
Epoch 6, Validation Loss: 506316.46
Epoch 7, Validation Loss: 455785.43
Epoch 8, Validation Loss: 479274.73
Epoch 9, Validation Loss: 1830764.43
Epoch 10, Validation Loss: 617360.01
Epoch 11, Validation Loss: 624843.02
Epoch 12, Validation Loss: 396752.18
Epoch 13, Validation Loss: 561413.43
Epoch 14, Validation Loss: 362575.21
Epoch 15, Validation Loss: 573894.39
Epoch 16, Validation Loss: 423855.20
Epoch 17, Validation Loss: 353895.04
Epoch 18, Validation Loss: 383910.57
Epoch 19, Validation Loss: 361177.79
Epoch 20, Validation Loss: 612085.78
Epoch 21, Validation Loss: 351211.70
Epoch 22, Validation Loss: 369439.86
Epoch 23, Validation Loss: 304853.19
Epoch 24, Validation Loss: 295741.54
Epoch 25, Validation Loss: 288538.73
Epoch 26, Validation Loss: 306635.38
Epoch 27,

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_st_50-10000.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:02<00:00,  1.82s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    72904.017814       1116.434110      0.006589
std     20744.559860       1941.529684      0.001487
min     37120.702194          0.000000      0.004511
25%     57189.760813          0.000000      0.005604
50%     72533.574198          0.000000      0.006099
75%     79193.075379       1966.710396      0.006837
max    118629.147466       8897.434940      0.012715
Number of infeasible solution: 35
